# Practical Exam: Loan Insights

EasyLoan offers a wide range of loan services, including personal loans, car loans, and mortgages.

EasyLoan offers loans to clients from Canada, United Kingdom and United States.

The analytics team wants to report performance across different geographic areas. They aim to identify areas of strength and weakness for the business strategy team.

They need your help to ensure the data is accessible and reliable before they start reporting.


**Database Schema**

The data you need is in the database named `lending`.

![database schema](lending_schema.png)

# Task 1 

The analytics team wants to use the `client` table to create a dashboard for client details. For them to proceed, they need to be sure the data is clean enough to use.

The `client` table below illustrates what the analytics team expects the data types and format to be.

Write a query that makes the `client` table match the description provided. Your query should not update the `client` table.

| Column Name       | Description                                                      |
|-------------------|------------------------------------------------------------------|
| client_id         | Unique integer (set by the database, can’t take any other value) |
| date\_of\_birth       | Date of birth of the client, as a date  (format: YYYY-MM-DD)                              |
| employment_status        | Current employment status of the client, either employed or unemployed, as a lower case string                              |
| country          | The country where the client resides, either USA, UK or CA, as an upper case string                      |

In [72]:
SELECT 
    client_id,
    CAST(date_of_birth AS DATE),
    CASE
        WHEN LOWER(employment_status) LIKE 'em%' THEN 'employed'
        WHEN LOWER(employment_status) LIKE 'un%' THEN 'unemployed'
        WHEN LOWER(employment_status) IN ('part-time', 'full-time') THEN 'employed'
        ELSE employment_status
    END AS employment_status,
    UPPER(country) AS country
FROM 
    client


,client_id,date_of_birth,employment_status,country
0,1,1963-07-08 00:00:00+00:00,unemployed,USA
1,2,1957-02-07 00:00:00+00:00,unemployed,UK
2,3,1993-02-21 00:00:00+00:00,employed,CA
3,4,1978-03-19 00:00:00+00:00,employed,CA
4,5,2000-10-02 00:00:00+00:00,employed,USA
...,...,...,...,...
295,296,1963-05-02 00:00:00+00:00,employed,CA
296,297,2004-08-25 00:00:00+00:00,employed,UK
297,298,1997-08-03 00:00:00+00:00,unemployed,UK
298,299,2004-08-16 00:00:00+00:00,unemployed,CA


**This solution should be correct, yet Datacamp is accepting the one above. After numerous attempts, I managed to resolve it, though there remains a space before 'DD'. Please feel free to address this issue.**

In [1]:
SELECT 
    client_id,
    SUBSTRING(TO_CHAR(date_of_birth::DATE, 'YYYY-MM- DD'), 1, 11) AS date_of_birth,
    CASE
        WHEN LOWER(employment_status) LIKE 'em%' THEN 'employed'
        WHEN LOWER(employment_status) LIKE 'un%' THEN 'unemployed'
        WHEN LOWER(employment_status) IN ('part-time', 'full-time') THEN 'employed'
        ELSE employment_status
    END AS employment_status,
    UPPER(country) AS country
FROM 
    client

,client_id,date_of_birth,employment_status,country
0,1,1963-07- 08,unemployed,USA
1,2,1957-02- 07,unemployed,UK
2,3,1993-02- 21,employed,CA
3,4,1978-03- 19,employed,CA
4,5,2000-10- 02,employed,USA
...,...,...,...,...
295,296,1963-05- 02,employed,CA
296,297,2004-08- 25,employed,UK
297,298,1997-08- 03,unemployed,UK
298,299,2004-08- 16,unemployed,CA


# Task 2

You have been told that there was a problem in the backend system as some of the `repayment_channel` values are missing. 

The missing values are critical to the analysis so they need to be filled in before proceeding.

Luckily, they have discovered a pattern in the missing values:

- Repayment higher than 4000 dollars should be made via `bank account`.
- Repayment lower than 1000 dollars should be made via `mail`.

Write a query that makes the `repayment` table match this criteria.

In [61]:
SELECT 
    repayment_id, 
    loan_id,
    repayment_date,
    repayment_amount,
    CASE 
        WHEN repayment_channel = '-' THEN
            CASE 
                WHEN repayment_amount > 4000 THEN 'bank account'
                WHEN repayment_amount < 1000 THEN 'mail'
                ELSE repayment_channel
            END
        ELSE repayment_channel
    END AS repayment_channel
FROM repayment;



,repayment_id,loan_id,repayment_date,repayment_amount,repayment_channel
0,1,357,2022-10-16 00:00:00+00:00,1675.83,bank account
1,2,805,2023-01-12 00:00:00+00:00,867.22,debit card
2,3,843,2022-06-02 00:00:00+00:00,718.83,phone
3,4,243,2022-12-26 00:00:00+00:00,1620.97,credit card
4,5,991,2023-03-18 00:00:00+00:00,2182.17,phone
...,...,...,...,...,...
1495,1496,192,2023-02-11 00:00:00+00:00,2506.47,debit card
1496,1497,999,2022-11-21 00:00:00+00:00,1324.61,bank account
1497,1498,966,2023-01-26 00:00:00+00:00,1375.81,debit card
1498,1499,20,2023-03-30 00:00:00+00:00,1231.25,debit card


# Task 3

Starting on January 1st, 2022, all US clients started to use an online system to sign contracts.

The analytics team wants to analyze the loans for US clients who used the new online system.

Write a query that returns the data for the analytics team. Your output should include `client_id`,`contract_date`, `principal_amount` and `loan_type` columns.

![database schema](lending_schema.png)

In [62]:
SELECT lo.client_id,
       con.contract_date,
       lo.principal_amount,
       lo.loan_type
FROM loan lo
JOIN contract con ON lo.contract_id = con.contract_id
JOIN client cli ON lo.client_id = cli.client_id
WHERE con.contract_date >= '2022-01-01' AND cli.country = 'USA';




,client_id,contract_date,principal_amount,loan_type
0,267,2022-03-08 00:00:00+00:00,179230,personal
1,50,2022-01-13 00:00:00+00:00,143729,mortgage
2,280,2022-01-02 00:00:00+00:00,171122,car
3,79,2022-01-24 00:00:00+00:00,43784,mortgage
4,245,2022-01-03 00:00:00+00:00,95003,mortgage
...,...,...,...,...
89,8,2022-01-13 00:00:00+00:00,148282,personal
90,232,2022-02-08 00:00:00+00:00,78386,personal
91,95,2022-03-20 00:00:00+00:00,39098,personal
92,123,2022-03-06 00:00:00+00:00,48213,car


# Task 4

The business strategy team is considering offering a more competitive rate to the US market. 

The analytic team want to compare the average interest rates offered by the company for the same loan type in different countries to determine if there are significant differences.

Write a query that returns the data for the analytics team. Your output should include `loan_type`, `country` and `avg_rate` columns.

![database schema](lending_schema.png)

In [63]:
SELECT lo.loan_type,
       cli.country,
       AVG(lo.interest_rate) AS avg_rate
FROM loan lo
JOIN client cli ON lo.client_id = cli.client_id
GROUP BY lo.loan_type, cli.country;


,loan_type,country,avg_rate
0,car,USA,0.103636
1,personal,CA,0.217253
2,personal,USA,0.202721
3,mortgage,CA,0.044068
4,mortgage,UK,0.042281
5,car,UK,0.122613
6,personal,UK,0.198738
7,mortgage,USA,0.043860
8,car,CA,0.112039
